In [46]:
import pandas as pd
import spacy

In [47]:
df = pd.read_csv('./../data/train.csv')
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \r\n\r\nThat...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \r\n\r\nUmm, theres no actual article ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [48]:
df.duplicated().sum(), df.isnull().sum()

(0,
 id               0
 comment_text     0
 toxic            0
 severe_toxic     0
 obscene          0
 threat           0
 insult           0
 identity_hate    0
 dtype: int64)

In [49]:
df['sentiment'] = df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].any(axis=1).astype(int)
df = df.drop(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"], axis=1)
df

,id,comment_text,sentiment
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0
159567,ffea4adeee384e90,You should be ashamed of yourself \r\n\r\nThat...,0
159568,ffee36eab5c267c9,"Spitzer \r\n\r\nUmm, theres no actual article ...",0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0


In [50]:
df['sentiment'].value_counts()

sentiment
0    143346
1     16225
Name: count, dtype: int64

In [51]:
nlp = spacy.load('en_core_web_sm')

In [52]:
def treat_comment(comment):
    spacy_comment = nlp(comment, disable=["parser", "tagger", "ner", "textcat"])
    treated_tokens = [w.text for w in spacy_comment if (w.is_alpha and not w.is_stop) or w.text.lower() in ["not", "no", "never", "none", "nothing", "don't"]]
    return " ".join(treated_tokens)

In [53]:
df_ones = df[df['sentiment'] == 1]
df_zeros = df[df['sentiment'] == 0]
df_zeros = df_zeros.sample(n=len(df[df['sentiment'] == 1]), random_state=42)
df = pd.concat([df_ones, df_zeros])

df['sentiment'].value_counts()

sentiment
1    16225
0    16225
Name: count, dtype: int64

In [ ]:
# import swifter
# df['comment_text'] = df['comment_text'].swifter.apply(treat_comment)
# df['comment_text'] = df['comment_text'].apply(treat_comment)
nlp = spacy.load('en_core_web_sm')

def treat_comment(comment, lp):
    spacy_comment = lp(comment, disable=["parser", "tagger", "ner", "textcat"])
    treated_tokens = [w.text for w in spacy_comment if (w.is_alpha and not w.is_stop) or w.text.lower() in ["not", "no", "never", "none", "nothing", "don't"]]
    return " ".join(treated_tokens)

from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

# df.apply(func)
df['comment_text'] = df['comment_text'].parallel_apply(treat_comment, lp=nlp)
# df.parallel_apply(func)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


TypeError: treat_comment() takes 2 positional arguments but 3 were given

In [55]:
Y = df["sentiment"]

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["comment_text"])
X, Y

(<32450x58865 sparse matrix of type '<class 'numpy.float64'>'
 	with 663458 stored elements in Compressed Sparse Row format>,
 6         1
 12        1
 16        1
 42        1
 43        1
          ..
 29862     0
 90273     0
 120264    0
 158396    0
 111060    0
 Name: sentiment, Length: 32450, dtype: int32)

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25,random_state=42)

In [57]:
from sklearn.linear_model import LogisticRegression

In [58]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [59]:
model.score(X_train, Y_train)

0.9398446809384887

In [60]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [61]:
y_pred = model.predict(X_test)

In [62]:
accuracy_score(Y_test, y_pred), confusion_matrix(Y_test, y_pred)

(0.8883273758165906,
 array([[3742,  299],
        [ 607, 3465]], dtype=int64))

In [67]:
commentaire= 'hello bobby'
test = treat_comment(commentaire)
test = vectorizer.transform([test])
y_pred = model.predict(test)
y_pred

hello bobby


array([0])